# This code file is to prepare for the data that we are going to use for the analysis

## What Files Are Given to Start with:

**us_cities_by_state_SEPT.2023.csv**: the names/urls of the articles we would be working with. This data is scraped from [this WikiPedia page](https://en.wikipedia.org/wiki/Category:Lists_of_cities_in_the_United_States_by_state) and the scraped result could be found [here](https://drive.google.com/file/d/1khouDmMaZyKo0y5WkFj4lu7g8o35x_98/view?usp=sharing)

**NST-EST2022-ALLDATA.csv**: this data is obtained from [here](https://www.census.gov/data/tables/time-series/demo/popest/2020s-state-total.html). I chose the full dataset instead of the excel file because the excel file is not in a table. I took out only the names of states and the estimated population of the states for the analysis

**US States by Region in Table - US Census Bureau.csv**: the original dataset *US States by Region - US Census Bureau.csv* comes from [this file](https://docs.google.com/spreadsheets/d/14Sjfd_u_7N9SSyQ7bmxfebF_2XpR8QamvmNntKDIQB0/edit?usp=sharing)

## What Tools Used in this Project:

**Wikimedia**:  to extract the page information of articles. [Sample code used in this project comes from here](https://drive.google.com/file/d/15UoE16s-IccCTOXREjU3xDIz07tlpyrl/view?usp=sharing)

**ORES model**: provides the peer reviewed score of each article based on the quality. [Sample code used in this project comes from here](https://drive.google.com/file/d/17C9xsmR9U3lJeD52UTbAedlHDetwYsxs/view?usp=sharing)

## What Steps Are Covered in this Code File:

**Scraping the page information of articles** - using the *Wikimedia API*

**Obtaining the assessed quality score for each article** - using the ORES model. A second attempt was performed after noting down which articles failed to retrieve its score, and only one failed again to retrieve its score on the second try which was removed. It is also worth noticing that there exist duplicates of rows of data which should be removed in order to perform the analysis

**Combining the datasets** - the previous steps result in a DataFrame with columns of article name, state, review id and score. Population (which we could obtain from the *NST-EST2022-ALLDATA.csv*) and regional data (which is in *US States by Region in Table - US Census Bureau.csv*) are required in the final dataset, which should contain in total 6 columns

### Scraping using Wikimedia

In [21]:
# !pip install openpyxl --upgrade
import pandas as pd
import json, time, urllib.parse
import requests
import warnings
warnings.simplefilter('ignore')

pop_est_2022 = pd.read_csv('NST-EST2022-ALLDATA.csv')
cities_by_state = pd.read_csv('us_cities_by_state_SEPT.2023.csv')

In [2]:
article_titles = list(cities_by_state['page_title'])

In [3]:
API_ENWIKIPEDIA_ENDPOINT = "https://en.wikipedia.org/w/api.php"

# We'll assume that there needs to be some throttling for these requests - we should always be nice to a free data resource
API_LATENCY_ASSUMED = 0.002       # Assuming roughly 2ms latency on the API and network
API_THROTTLE_WAIT = (1.0/100.0)-API_LATENCY_ASSUMED

# When making automated requests we should include something that is unique to the person making the request
# This should include an email - your UW email would be good to put in there
REQUEST_HEADERS = {
    'User-Agent': 'hww1999@uw.edu, University of Washington, MSDS DATA 512 - AUTUMN 2023',
}

# This is just a list of English Wikipedia article titles that we can use for example requests

ARTICLE_TITLES = article_titles

# This is a string of additional page properties that can be returned see the Info documentation for
# what can be included. If you don't want any this can simply be the empty string
PAGEINFO_EXTENDED_PROPERTIES = "talkid|url|watched|watchers"
#PAGEINFO_EXTENDED_PROPERTIES = ""

# This template lists the basic parameters for making this
PAGEINFO_PARAMS_TEMPLATE = {
    "action": "query",
    "format": "json",
    "titles": "",           # to simplify this should be a single page title at a time
    "prop": "info",
    "inprop": PAGEINFO_EXTENDED_PROPERTIES
}


In [4]:
def request_pageinfo_per_article(article_title = None, 
                                 endpoint_url = API_ENWIKIPEDIA_ENDPOINT, 
                                 request_template = PAGEINFO_PARAMS_TEMPLATE,
                                 headers = REQUEST_HEADERS):
    
    # article title can be as a parameter to the call or in the request_template
    if article_title:
        request_template['titles'] = article_title

    if not request_template['titles']:
        raise Exception("Must supply an article title to make a pageinfo request.")

    # make the request
    try:
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        response = requests.get(endpoint_url, headers=headers, params=request_template)
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response


### Handled exceptions here to prevent failure to retrieve

One intermediary data file generated here for backup purpose.

In [5]:
cities_by_state_articles = {"articles":[]}
for i in range(len(ARTICLE_TITLES)):
    print("Getting page information for: ",ARTICLE_TITLES[i])
    views = request_pageinfo_per_article(ARTICLE_TITLES[i])
    try:
        pid = list(views['query']['pages'].keys())[0]
        cities_by_state_articles["articles"].append(views['query']['pages'][pid])
    except Exception as e:
        print(e)
with open("cities_by_state.json", "a") as outfile:
        outfile.write(json.dumps(cities_by_state_articles,indent=4))

Getting page information for:  Abbeville, Alabama
Getting page information for:  Adamsville, Alabama
Getting page information for:  Addison, Alabama
Getting page information for:  Akron, Alabama
Getting page information for:  Alabaster, Alabama
Getting page information for:  Albertville, Alabama
Getting page information for:  Alexander City, Alabama
Getting page information for:  Aliceville, Alabama
Getting page information for:  Allgood, Alabama
Getting page information for:  Altoona, Alabama
Getting page information for:  Andalusia, Alabama
Getting page information for:  Anderson, Lauderdale County, Alabama
Getting page information for:  Anniston, Alabama
Getting page information for:  Arab, Alabama
Getting page information for:  Ardmore, Alabama
Getting page information for:  Argo, Alabama
Getting page information for:  Ariton, Alabama
Getting page information for:  Arley, Alabama
Getting page information for:  Ashford, Alabama
Getting page information for:  Ashland, Alabama
Getting

Getting page information for:  Glen Allen, Alabama
Getting page information for:  Glencoe, Alabama
Getting page information for:  Glenwood, Alabama
Getting page information for:  Goldville, Alabama
Getting page information for:  Good Hope, Alabama
Getting page information for:  Goodwater, Alabama
Getting page information for:  Gordo, Alabama
Getting page information for:  Gordon, Alabama
Getting page information for:  Gordonville, Alabama
Getting page information for:  Goshen, Alabama
Getting page information for:  Grant, Alabama
Getting page information for:  Graysville, Alabama
Getting page information for:  Greensboro, Alabama
Getting page information for:  Greenville, Alabama
Getting page information for:  Grimes, Alabama
Getting page information for:  Grove Hill, Alabama
Getting page information for:  Guin, Alabama
Getting page information for:  Gulf Shores, Alabama
Getting page information for:  Guntersville, Alabama
Getting page information for:  Gurley, Alabama
Getting page inf

Getting page information for:  Piedmont, Alabama
Getting page information for:  Pike Road, Alabama
Getting page information for:  Pinckard, Alabama
Getting page information for:  Pine Apple, Alabama
Getting page information for:  Pine Hill, Alabama
Getting page information for:  Pine Ridge, Alabama
Getting page information for:  Pinson, Alabama
Getting page information for:  Pisgah, Alabama
Getting page information for:  Pleasant Grove, Alabama
Getting page information for:  Pleasant Groves, Alabama
Getting page information for:  Pollard, Alabama
Getting page information for:  Powell, Alabama
Getting page information for:  Prattville, Alabama
Getting page information for:  Priceville, Alabama
Getting page information for:  Prichard, Alabama
Getting page information for:  Providence, Alabama
Getting page information for:  Ragland, Alabama
Getting page information for:  Rainbow City, Alabama
Getting page information for:  Rainsville, Alabama
Getting page information for:  Ranburne, Alaba

Getting page information for:  Berry, Alabama
Getting page information for:  Bessemer, Alabama
Getting page information for:  Billingsley, Alabama
Getting page information for:  Birmingham, Alabama
Getting page information for:  Black, Alabama
Getting page information for:  Blountsville, Alabama
Getting page information for:  Blue Springs, Alabama
Getting page information for:  Boaz, Alabama
Getting page information for:  Boligee, Alabama
Getting page information for:  Bon Air, Alabama
Getting page information for:  Brantley, Alabama
Getting page information for:  Brent, Alabama
Getting page information for:  Brewton, Alabama
Getting page information for:  Bridgeport, Alabama
Getting page information for:  Brighton, Alabama
Getting page information for:  Brilliant, Alabama
Getting page information for:  Brookside, Alabama
Getting page information for:  Brookwood, Alabama
Getting page information for:  Brundidge, Alabama
Getting page information for:  Butler, Alabama
Getting page inform

Getting page information for:  Hobson City, Alabama
Getting page information for:  Hodges, Alabama
Getting page information for:  Hokes Bluff, Alabama
Getting page information for:  Holly Pond, Alabama
Getting page information for:  Hollywood, Alabama
Getting page information for:  Homewood, Alabama
Getting page information for:  Hoover, Alabama
Getting page information for:  Horn Hill, Alabama
Getting page information for:  Hueytown, Alabama
Getting page information for:  Huntsville, Alabama
Getting page information for:  Hurtsboro, Alabama
Getting page information for:  Hytop, Alabama
Getting page information for:  Ider, Alabama
Getting page information for:  Indian Springs Village, Alabama
Getting page information for:  Irondale, Alabama
Getting page information for:  Jackson, Alabama
Getting page information for:  Jackson's Gap, Alabama
Getting page information for:  Jacksonville, Alabama
Getting page information for:  Jasper, Alabama
Getting page information for:  Jemison, Alabama

...

Getting page information for:  North Bend, Washington
Getting page information for:  North Bonneville, Washington
Getting page information for:  Oak Harbor, Washington
Getting page information for:  Oakville, Washington
Getting page information for:  Ocean Shores, Washington
Getting page information for:  Okanogan, Washington
Getting page information for:  Olympia, Washington
Getting page information for:  Omak, Washington
Getting page information for:  Oroville, Washington
Getting page information for:  Orting, Washington
Getting page information for:  Othello, Washington
Getting page information for:  Pacific, Washington
Getting page information for:  Palouse, Washington
Getting page information for:  Pasco, Washington
Getting page information for:  Pateros, Washington
Getting page information for:  Pomeroy, Washington
Getting page information for:  Port Angeles, Washington
Getting page information for:  Port Orchard, Washington
Getting page information for:  Port Townsend, Washingto

Getting page information for:  Huntington, West Virginia
Getting page information for:  Parkersburg, West Virginia
Getting page information for:  Morgantown, West Virginia
Getting page information for:  Wheeling, West Virginia
Getting page information for:  Martinsburg, West Virginia
Getting page information for:  Weirton, West Virginia
Getting page information for:  Fairmont, West Virginia
Getting page information for:  Beckley, West Virginia
Getting page information for:  Clarksburg, West Virginia
Getting page information for:  South Charleston, West Virginia
Getting page information for:  St. Albans, West Virginia
Getting page information for:  Vienna, West Virginia
Getting page information for:  Bluefield, West Virginia
Getting page information for:  Bridgeport, West Virginia
Getting page information for:  Oak Hill, West Virginia
Getting page information for:  Moundsville, West Virginia
Getting page information for:  Dunbar, West Virginia
Getting page information for:  Hurricane, W

Getting page information for:  Anmoore, West Virginia
Getting page information for:  Hartford City, West Virginia
Getting page information for:  Franklin, West Virginia
Getting page information for:  Cowen, West Virginia
Getting page information for:  Grantsville, West Virginia
Getting page information for:  Pratt, West Virginia
Getting page information for:  Clearview, West Virginia
Getting page information for:  Rowlesburg, West Virginia
Getting page information for:  Peterstown, West Virginia
Getting page information for:  West Milford, West Virginia
Getting page information for:  Capon Bridge, West Virginia
Getting page information for:  Union, West Virginia
Getting page information for:  Delbarton, West Virginia
Getting page information for:  Paw Paw, West Virginia
Getting page information for:  Jane Lew, West Virginia
Getting page information for:  Farmington, West Virginia
Getting page information for:  Matewan, West Virginia
Getting page information for:  Burnsville, West Virgi

Getting page information for:  Hillsboro, Wisconsin
Getting page information for:  Horicon, Wisconsin
Getting page information for:  Hudson, Wisconsin
Getting page information for:  Hurley, Wisconsin
Getting page information for:  Independence, Wisconsin
Getting page information for:  Janesville, Wisconsin
Getting page information for:  Jefferson, Wisconsin
Getting page information for:  Juneau, Wisconsin
Getting page information for:  Kaukauna, Wisconsin
Getting page information for:  Kenosha, Wisconsin
Getting page information for:  Kewaunee, Wisconsin
Getting page information for:  Kiel, Wisconsin
Getting page information for:  La Crosse, Wisconsin
Getting page information for:  Ladysmith, Wisconsin
Getting page information for:  Lake Geneva, Wisconsin
Getting page information for:  Lake Mills, Wisconsin
Getting page information for:  Lancaster, Wisconsin
Getting page information for:  Lodi, Wisconsin
Getting page information for:  Loyal, Wisconsin
Getting page information for:  Mad

Getting page information for:  Granger, Wyoming
Getting page information for:  Green River, Wyoming
Getting page information for:  Greybull, Wyoming
Getting page information for:  Guernsey, Wyoming
Getting page information for:  Hanna, Wyoming
Getting page information for:  Hartville, Wyoming
Getting page information for:  Hudson, Wyoming
Getting page information for:  Hulett, Wyoming
Getting page information for:  Jackson, Wyoming
Getting page information for:  Kaycee, Wyoming
Getting page information for:  Kemmerer, Wyoming
Getting page information for:  Kirby, Wyoming
Getting page information for:  La Barge, Wyoming
Getting page information for:  La Grange, Wyoming
Getting page information for:  Lander, Wyoming
Getting page information for:  Laramie, Wyoming
Getting page information for:  Lingle, Wyoming
Getting page information for:  Lost Springs, Wyoming
Getting page information for:  Lovell, Wyoming
Getting page information for:  Lusk, Wyoming
Getting page information for:  Lyman

In [22]:
tmp = pd.read_json('cities_by_state.json')['articles']
tmp

0        {'pageid': 104730, 'ns': 0, 'title': 'Abbevill...
1        {'pageid': 104761, 'ns': 0, 'title': 'Adamsvil...
2        {'pageid': 105188, 'ns': 0, 'title': 'Addison,...
3        {'pageid': 104726, 'ns': 0, 'title': 'Akron, A...
4        {'pageid': 105109, 'ns': 0, 'title': 'Alabaste...
                               ...                        
22152    {'pageid': 140221, 'ns': 0, 'title': 'Wamsutte...
22153    {'pageid': 140185, 'ns': 0, 'title': 'Wheatlan...
22154    {'pageid': 140245, 'ns': 0, 'title': 'Worland,...
22155    {'pageid': 140070, 'ns': 0, 'title': 'Wright, ...
22156    {'pageid': 140112, 'ns': 0, 'title': 'Yoder, W...
Name: articles, Length: 22157, dtype: object

In [23]:
tmp = pd.json_normalize(tmp)
tmp

,pageid,ns,title,contentmodel,pagelanguage,pagelanguagehtmlcode,pagelanguagedir,touched,lastrevid,length,talkid,fullurl,editurl,canonicalurl,watchers,redirect,new
0,104730,0,"Abbeville, Alabama",wikitext,en,en,ltr,2023-10-09T13:09:29Z,1171163550,24706,281244.0,"https://en.wikipedia.org/wiki/Abbeville,_Alabama",https://en.wikipedia.org/w/index.php?title=Abb...,"https://en.wikipedia.org/wiki/Abbeville,_Alabama",NaN,NaN,NaN
1,104761,0,"Adamsville, Alabama",wikitext,en,en,ltr,2023-10-09T13:09:29Z,1177621427,18040,281272.0,"https://en.wikipedia.org/wiki/Adamsville,_Alabama",https://en.wikipedia.org/w/index.php?title=Ada...,"https://en.wikipedia.org/wiki/Adamsville,_Alabama",NaN,NaN,NaN
2,105188,0,"Addison, Alabama",wikitext,en,en,ltr,2023-10-09T13:09:29Z,1168359898,13309,281517.0,"https://en.wikipedia.org/wiki/Addison,_Alabama",https://en.wikipedia.org/w/index.php?title=Add...,"https://en.wikipedia.org/wiki/Addison,_Alabama",NaN,NaN,NaN
3,104726,0,"Akron, Alabama",wikitext,en,en,ltr,2023-10-09T13:09:29Z,1165909508,11710,281240.0,"https://en.wikipedia.org/wiki/Akron,_Alabama",https://en.wikipedia.org/w/index.php?title=Akr...,"https://en.wikipedia.org/wiki/Akron,_Alabama",NaN,NaN,NaN
4,105109,0,"Alabaster, Alabama",wikitext,en,en,ltr,2023-10-09T13:09:29Z,1179139816,20343,281444.0,"https://en.wikipedia.org/wiki/Alabaster,_Alabama",https://en.wikipedia.org/w/index.php?title=Ala...,"https://en.wikipedia.org/wiki/Alabaster,_Alabama",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22152,140221,0,"Wamsutter, Wyoming",wikitext,en,en,ltr,2023-10-09T13:09:55Z,1169591845,15315,7761801.0,"https://en.wikipedia.org/wiki/Wamsutter,_Wyoming",https://en.wikipedia.org/w/index.php?title=Wam...,"https://en.wikipedia.org/wiki/Wamsutter,_Wyoming",NaN,NaN,NaN
22153,140185,0,"Wheatland, Wyoming",wikitext,en,en,ltr,2023-10-09T13:09:55Z,1176370621,20494,10817367.0,"https://en.wikipedia.org/wiki/Wheatland,_Wyoming",https://en.wikipedia.org/w/index.php?title=Whe...,"https://en.wikipedia.org/wiki/Wheatland,_Wyoming",NaN,NaN,NaN
22154,140245,0,"Worland, Wyoming",wikitext,en,en,ltr,2023-10-09T13:09:55Z,1166347917,19443,10265719.0,"https://en.wikipedia.org/wiki/Worland,_Wyoming",https://en.wikipedia.org/w/index.php?title=Wor...,"https://en.wikipedia.org/wiki/Worland,_Wyoming",NaN,NaN,NaN
22155,140070,0,"Wright, Wyoming",wikitext,en,en,ltr,2023-10-09T13:09:55Z,1166334449,12129,10717664.0,"https://en.wikipedia.org/wiki/Wright,_Wyoming",https://en.wikipedia.org/w/index.php?title=Wri...,"https://en.wikipedia.org/wiki/Wright,_Wyoming",NaN,NaN,NaN


In [24]:
tmp.columns

Index(['pageid', 'ns', 'title', 'contentmodel', 'pagelanguage',
       'pagelanguagehtmlcode', 'pagelanguagedir', 'touched', 'lastrevid',
       'length', 'talkid', 'fullurl', 'editurl', 'canonicalurl', 'watchers',
       'redirect', 'new'],
      dtype='object')

In [25]:
df_eval = tmp[['pageid', 'title', 'lastrevid', 'talkid']]

### Evaluation using ORES model

In [46]:
API_ORES_LIFTWING_ENDPOINT = "https://api.wikimedia.org/service/lw/inference/v1/models/{model_name}:predict"
API_ORES_EN_QUALITY_MODEL = "enwiki-articlequality"

#
#    The throttling rate is a function of the Access token that you are granted when you request the token. The constants
#    come from dissecting the token and getting the rate limits from the granted token. An example of that is below.
#
API_LATENCY_ASSUMED = 0.002       # Assuming roughly 2ms latency on the API and network
API_THROTTLE_WAIT = (60.0/5000.0)-API_LATENCY_ASSUMED

#    When making automated requests we should include something that is unique to the person making the request
#    This should include an email - your UW email would be good to put in there
#    
#    Because all LiftWing API requests require some form of authentication, you need to provide your access token
#    as part of the header too
#
REQUEST_HEADER_TEMPLATE = {
    'User-Agent': "<{email_address}>, University of Washington, MSDS DATA 512 - AUTUMN 2023",
    'Content-Type': 'application/json',
    'Authorization': "Bearer {access_token}"
}

#
#    This is a template for the parameters that we need to supply in the headers of an API request
#

REQUEST_HEADER_PARAMS_TEMPLATE = {
    'email_address' : "",         # your email address should go here
    'access_token'  : ""          # the access token you create will need to go here
}

#
#    A dictionary of English Wikipedia article titles (keys) and sample revision IDs that can be used for this ORES scoring example
#
ARTICLE_REVISIONS = {'Abbeville, Alabama': 1171163550}
# { 'Bison':1085687913 , 'Northern flicker':1086582504 , 'Red squirrel':1083787665 , 'Chinook salmon':1085406228 , 'Horseshoe bat':1060601936 }

#
#    This is a template of the data required as a payload when making a scoring request of the ORES model
#
ORES_REQUEST_DATA_TEMPLATE = {
    "lang":        "en",     # required that its english - we're scoring English Wikipedia revisions
    "rev_id":      "",       # this request requires a revision id
    "features":    True
}



### Personal access token is saved in a local file

In [ ]:
# input personal information
# access token saved as a local file

token_info = pd.read_csv('WikiAPIToken.csv')
ACCESS_TOKEN = token_info['Access token'][0]
USERNAME = "hww1999@uw.edu"

In [47]:
#########
#
#    PROCEDURES/FUNCTIONS
#

def request_ores_score_per_article(article_revid = None, email_address=None, access_token=None,
                                   endpoint_url = API_ORES_LIFTWING_ENDPOINT, 
                                   model_name = API_ORES_EN_QUALITY_MODEL, 
                                   request_data = ORES_REQUEST_DATA_TEMPLATE, 
                                   header_format = REQUEST_HEADER_TEMPLATE, 
                                   header_params = REQUEST_HEADER_PARAMS_TEMPLATE):
    
    #    Make sure we have an article revision id, email and token
    #    This approach prioritizes the parameters passed in when making the call
    if article_revid:
        request_data['rev_id'] = article_revid
    if email_address:
        header_params['email_address'] = email_address
    if access_token:
        header_params['access_token'] = access_token
    
    #   Making a request requires a revision id - an email address - and the access token
    if not request_data['rev_id']:
        raise Exception("Must provide an article revision id (rev_id) to score articles")
    if not header_params['email_address']:
        raise Exception("Must provide an 'email_address' value")
    if not header_params['access_token']:
        raise Exception("Must provide an 'access_token' value")
    
    # Create the request URL with the specified model parameter - default is a article quality score request
    request_url = endpoint_url.format(model_name=model_name)
    
    # Create a compliant request header from the template and the supplied parameters
    headers = dict()
    for key in header_format.keys():
        headers[str(key)] = header_format[key].format(**header_params)
    
    # make the request
    try:
        # we'll wait first, to make sure we don't exceed the limit in the situation where an exception
        # occurs during the request processing - throttling is always a good practice with a free data
        # source like ORES - or other community sources
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        #response = requests.get(request_url, headers=headers)
        response = requests.post(request_url, headers=headers, data=json.dumps(request_data))
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response


### Below is the first attempt with 107 failures

In [67]:
score_pred = []
from tqdm import tqdm
for i in tqdm(range(df_eval.shape[0])):
    
    article_title = df_eval.iloc[i]['title']
    article_revid = df_eval.iloc[i]['lastrevid']
    
#     print(f"Getting LiftWing ORES scores for '{article_title}' with revid: {article_revid:d}")
    try:
    
        score = request_ores_score_per_article(article_revid=int(article_revid),
                                               email_address=USERNAME,
                                               access_token=ACCESS_TOKEN)

        pred = score['enwiki']['scores'][str(article_revid)]['articlequality']['score']['prediction']
#         print(article_title + ': ' + pred)
        score_pred.append(pred)
    except Exception as e:
        print(e)
        print(article_title + ': Unknown')
        score_pred.append('REQUEST FAIL')
score_pred

  2%|▏         | 486/22157 [04:52<3:01:14,  1.99it/s]

'enwiki'
Auburn, Alabama: Unknown


  3%|▎         | 690/22157 [06:35<2:48:06,  2.13it/s]

Expecting value: line 1 column 1 (char 0)
'NoneType' object is not subscriptable
Kennedy, Alabama: Unknown


 42%|████▏     | 9335/22157 [1:53:56<1:25:59,  2.49it/s] 

...

('Connection aborted.', ConnectionResetError(10054, '远程主机强迫关闭了一个现有的连接。', None, 10054, None))
'NoneType' object is not subscriptable
Clarion, Pennsylvania: Unknown


 74%|███████▍  | 16402/22157 [3:50:00<2:31:30,  1.58s/it]

Expecting value: line 1 column 1 (char 0)
'NoneType' object is not subscriptable
Conneaut Township, Erie County, Pennsylvania: Unknown


100%|██████████| 22157/22157 [5:24:13<00:00,  1.14it/s]  


['C',
 'C',
 'C',
 'GA',
 'C',
 'C',
 'GA',
 'GA',
 'C',
 'C',
 'C',
 'Stub',
 'C',
 'C',
 'GA',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'Start',
 'Start',
 'C',
 'Start',
 'C',
 'C',
 'C',
 'GA',
 'C',
 'Start',
 'C',
 'C',
 'Start',
 'C',
 'C',
 'Start',
 'B',
 'Start',
 'GA',
 'Start',
 'GA',
 'C',
 'C',
 'C',
 'GA',
 'GA',
 'GA',
 'C',
 'GA',
 'C',
 'C',
 'C',
 'C',
 'C',
 'GA',
 'C',
 'C',
 'Start',
 'Start',
 'C',
 'Start',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'GA',
 'C',
 'GA',
 'C',
 'Start',
 'C',
 'C',
 'Start',
 'C',
 'Start',
 'C',
 'Start',
 'C',
 'GA',
 'C',
 'C',
 'C',
 'GA',
 'C',
 'Start',
 'GA',
 'C',
 'GA',
 'C',
 'C',
 'B',
 'Start',
 'C',
 'GA',
 'FA',
 'B',
 'C',
 'C',
 'C',
 'C',
 'B',
 'Start',
 'Start',
 'C',
 'B',
 'C',
 'GA',
 'Start',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'Start',
 'C',
 'C',
 'C',
 'C',
 'C',
 'Start',
 'GA',
 'C',
 'GA',
 'C',
 'C',
 'C',
 'C',
 'C',
 'GA',
 'GA',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 '

### Copy and paste the failed attemps and their errors to have a second try on the failures

### We could also obtain de failed attempts from the df_eval where score='FAILED REQUEST'

'enwiki'
Auburn, Alabama: Unknown

Expecting value: line 1 column 1 (char 0)
'NoneType' object is not subscriptable
Kennedy, Alabama: Unknown

'enwiki'
Huntington Woods, Michigan: Unknown

'enwiki'
Huron Township, Michigan: Unknown

'enwiki'
Huron Charter Township, Michigan: Unknown

'enwiki'
Ida Township, Michigan: Unknown

'enwiki'
Imlay Township, Michigan: Unknown

'enwiki'
Imlay City, Michigan: Unknown

'enwiki'
Independence Charter Township, Michigan: Unknown

'enwiki'
Indianfields Township, Michigan: Unknown

'enwiki'
Ingallston Township, Michigan: Unknown

'enwiki'
Ingersoll Township, Michigan: Unknown

'enwiki'
Ingham Township, Michigan: Unknown

'enwiki'
Inkster, Michigan: Unknown

'enwiki'
Inland Township, Michigan: Unknown

'enwiki'
Interior Township, Michigan: Unknown

'enwiki'
Inverness Township, Michigan: Unknown

'enwiki'
Inwood Township, Michigan: Unknown

'enwiki'
Ionia, Michigan: Unknown

'enwiki'
Ionia Township, Michigan: Unknown

'enwiki'
Iosco Township, Michigan: Unknown

'enwiki'
Ira Township, Michigan: Unknown

'enwiki'
Iron Mountain, Michigan: Unknown

'enwiki'
Iron River, Michigan: Unknown

'enwiki'
Iron River Township, Michigan: Unknown

'enwiki'
Ironwood, Michigan: Unknown

'enwiki'
Ironwood Charter Township, Michigan: Unknown

'enwiki'
Irving Township, Michigan: Unknown

'enwiki'
Isabella Township, Michigan: Unknown

'enwiki'
Ishpeming, Michigan: Unknown

'enwiki'
Ishpeming Township, Michigan: Unknown

'enwiki'
Ithaca, Michigan: Unknown

'enwiki'
Jackson, Michigan: Unknown

'enwiki'
James Township, Michigan: Unknown

'enwiki'
Jamestown Charter Township, Michigan: Unknown

'enwiki'
Jasper Township, Michigan: Unknown

'enwiki'
Jefferson Township, Cass County, Michigan: Unknown

'enwiki'
Jefferson Township, Hillsdale County, Michigan: Unknown

'enwiki'
Jerome Township, Michigan: Unknown

'enwiki'
Johnstown Township, Michigan: Unknown

'enwiki'
Jonesfield Township, Michigan: Unknown

'enwiki'
Jonesville, Michigan: Unknown

'enwiki'
Jordan Township, Michigan: Unknown

'enwiki'
Joyfield Township, Michigan: Unknown

'enwiki'
Juniata Township, Michigan: Unknown

'enwiki'
Kalamazoo, Michigan: Unknown

'enwiki'
Kalamazoo Township, Michigan: Unknown

'enwiki'
Kalamo Township, Michigan: Unknown

'enwiki'
Kaleva, Michigan: Unknown

'enwiki'
Kalkaska, Michigan: Unknown

'enwiki'
Kalkaska Township, Michigan: Unknown

'enwiki'
Kasson Township, Michigan: Unknown

'enwiki'
Kawkawlin Township, Michigan: Unknown

'enwiki'
Kearney Township, Michigan: Unknown

'enwiki'
Keego Harbor, Michigan: Unknown

'enwiki'
Keeler Township, Michigan: Unknown

'enwiki'
Keene Township, Michigan: Unknown

'enwiki'
Kenockee Township, Michigan: Unknown

'enwiki'
Kent City, Michigan: Unknown

'enwiki'
Kentwood, Michigan: Unknown

'enwiki'
Kimball Township, Michigan: Unknown

'enwiki'
Kinde, Michigan: Unknown

'enwiki'
Kinderhook Township, Michigan: Unknown

'enwiki'
Kingsford, Michigan: Unknown

'enwiki'
Kingsley, Michigan: Unknown

'enwiki'
Kingston, Michigan: Unknown

'enwiki'
Kingston Township, Michigan: Unknown

'enwiki'
Kinross Charter Township, Michigan: Unknown

'enwiki'
Klacking Township, Michigan: Unknown

'enwiki'
Kochville Township, Michigan: Unknown

'enwiki'
Koehler Township, Michigan: Unknown

'enwiki'
Koylton Township, Michigan: Unknown

'enwiki'
Krakow Township, Michigan: Unknown

'enwiki'
Lafayette Township, Michigan: Unknown

'enwiki'
LaGrange Township, Michigan: Unknown

'enwiki'
Laingsburg, Michigan: Unknown

'enwiki'
Laird Township, Michigan: Unknown
 
'enwiki'
Lake Township, Benzie County, Michigan: Unknown

'enwiki'
Lake Charter Township, Michigan: Unknown

'enwiki'
Lake Township, Huron County, Michigan: Unknown

'enwiki'
Lake Township, Lake County, Michigan: Unknown

'enwiki'
Lake Township, Menominee County, Michigan: Unknown

'enwiki'
Lake Township, Missaukee County, Michigan: Unknown

'enwiki'
Lake Township, Roscommon County, Michigan: Unknown

'enwiki'
Lake Angelus, Michigan: Unknown

'enwiki'
Lake Ann, Michigan: Unknown

'enwiki'
Lake City, Michigan: Unknown

'enwiki'
Lakefield Township, Luce County, Michigan: Unknown

'enwiki'
Lakefield Township, Saginaw County, Michigan: Unknown

'enwiki'
Lake Isabella, Michigan: Unknown

'enwiki'
Lake Linden, Michigan: Unknown

'enwiki'
Lake Odessa, Michigan: Unknown
 
'enwiki'
Lake Orion, Michigan: Unknown

'enwiki'
Laketon Township, Michigan: Unknown

'enwiki'
Laketown Township, Michigan: Unknown

'enwiki'
Lakeview, Michigan: Unknown

'enwiki'
Lakewood Club, Michigan: Unknown

'enwiki'
Lamotte Township, Michigan: Unknown

'enwiki'
L'Anse, Michigan: Unknown

'enwiki'
L'Anse Township, Michigan: Unknown

'enwiki'
Lansing, Michigan: Unknown

'enwiki'
Lansing Charter Township, Michigan: Unknown

'enwiki'
Lapeer, Michigan: Unknown
 
Expecting value: line 1 column 1 (char 0)
'NoneType' object is not subscriptable
Pittsfield Charter Township, Michigan: Unknown
 
('Connection aborted.', ConnectionResetError(10054, '远程主机强迫关闭了一个现有的连接。', None, 10054, None))
'NoneType' object is not subscriptable
Macks Creek, Missouri: Unknown

('Connection aborted.', ConnectionResetError(10054, '远程主机强迫关闭了一个现有的连接。', None, 10054, None))
'NoneType' object is not subscriptable
Clarion, Pennsylvania: Unknown

Expecting value: line 1 column 1 (char 0)
'NoneType' object is not subscriptable
Conneaut Township, Erie County, Pennsylvania: Unknown

### The second intermediary dataset generated for back up purpose

In [196]:
# df_eval['score_pred'] = pd.Series(score_pred)
# df_eval.to_csv('score_first_attemp.csv', index=False)
df_eval = pd.read_csv('score_first_attemp.csv')

### At this time, I realized I forgot to include the state column from the given file, so added in here

In [206]:
df_eval['state'] = cities_by_state['state']

In [207]:
df_eval

,title,lastrevid,score_pred,state
0,"Abbeville, Alabama",1171163550,C,Alabama
1,"Adamsville, Alabama",1177621427,C,Alabama
2,"Addison, Alabama",1168359898,C,Alabama
3,"Akron, Alabama",1165909508,GA,Alabama
4,"Alabaster, Alabama",1179139816,C,Alabama
...,...,...,...,...
22152,"Wamsutter, Wyoming",1169591845,GA,Wyoming
22153,"Wheatland, Wyoming",1176370621,GA,Wyoming
22154,"Worland, Wyoming",1166347917,GA,Wyoming
22155,"Wright, Wyoming",1166334449,GA,Wyoming


In [208]:
df_failed_first_attempt = df_eval[df_eval['score_pred']=='REQUEST FAIL'][['title', 'lastrevid']]
df_failed_first_attempt = df_failed_first_attempt.reset_index()

### This attempt fixed 106 failed retrievals among 107

In [209]:
second_attempt = []
for i in range(df_failed_first_attempt.shape[0]):
    
    article_title = df_failed_first_attempt.iloc[i]['title']
    article_revid = df_failed_first_attempt.iloc[i]['lastrevid']
    
#     print(f"Getting LiftWing ORES scores for '{article_title}' with revid: {article_revid:d}")
    try:
    
        score = request_ores_score_per_article(article_revid=int(article_revid),
                                               email_address=USERNAME,
                                               access_token=ACCESS_TOKEN)
#         print(score)
        second_attempt.append(score['enwiki']['scores'][str(article_revid)]['articlequality']['score']['prediction'])
#         pred = score['enwiki']['scores'][str(article_revid)]['articlequality']['score']['prediction']
#         print(article_title + ': ' + pred)
#         second_attempt.append(pred)
    except Exception as e:
#         print(e)
#         print(article_title + ': Unknown')
        second_attempt.append('REQUEST FAIL')
df_failed_first_attempt['score'] = pd.Series(second_attempt)

In [213]:
# df_failed_first_attempt.reset_index()
# df_failed_first_attempt['score'] = pd.Series(second_attempt)
df_failed_first_attempt[df_failed_first_attempt['score']=='REQUEST FAIL']

,index,title,lastrevid,score
0,485,"Auburn, Alabama",1179546333,REQUEST FAIL


In [211]:
for i in range(df_failed_first_attempt.shape[0]):
    currtitle = df_failed_first_attempt.iloc[i]['title']
    currrevid = df_failed_first_attempt.iloc[i]['lastrevid']
    currscore = df_failed_first_attempt.iloc[i]['score']
    idx = df_eval[df_eval['title']==currtitle][df_eval['lastrevid']==currrevid].index
    df_eval.loc[idx,['score_pred']] = currscore

### Dropped duplicates at this point

In [212]:
df_eval = df_eval.drop_duplicates()
df_eval.shape

(21526, 4)

In [224]:
df_eval = df_eval.reset_index()

### We found that the same article but with a different review id is there with score already, so removed the one that failed

In [230]:
df_eval[df_eval['score_pred']=='REQUEST FAIL']

,index,title,lastrevid,score_pred,state
461,485,"Auburn, Alabama",1179546333,REQUEST FAIL,Alabama


In [231]:
df_eval[df_eval['title']=='Auburn, Alabama']

,index,title,lastrevid,score_pred,state
24,24,"Auburn, Alabama",1177335918,C,Alabama
461,485,"Auburn, Alabama",1179546333,REQUEST FAIL,Alabama


### The third intermediary data file generated which should overwrite the score_first_attemp.csv given that it only filled in the scores that failed to retrieve at the first try

In [259]:
df_eval = df_eval.drop([461])
df_eval = df_eval.reset_index()
df_eval.to_csv('updated_score.csv', index=False)

### Combining the datasets

In [287]:
df_eval = pd.read_csv('updated_score.csv')
df_eval = df_eval.reset_index()

In [288]:
df_eval = df_eval.drop(columns=['level_0', 'index'])
df_eval

,title,lastrevid,score_pred,state
0,"Abbeville, Alabama",1171163550,C,Alabama
1,"Adamsville, Alabama",1177621427,C,Alabama
2,"Addison, Alabama",1168359898,C,Alabama
3,"Akron, Alabama",1165909508,GA,Alabama
4,"Alabaster, Alabama",1179139816,C,Alabama
...,...,...,...,...
21520,"Wamsutter, Wyoming",1169591845,GA,Wyoming
21521,"Wheatland, Wyoming",1176370621,GA,Wyoming
21522,"Worland, Wyoming",1166347917,GA,Wyoming
21523,"Wright, Wyoming",1166334449,GA,Wyoming


### Noted there is no Connecticut or Nebraska from the original scraped dataset

In [289]:
len(df_eval['state'].unique()) # no Connecticut or Nebraska

48

In [290]:
df_eval['state'].unique() # need to replace '_' with space, and 'Georgia_(U.S._state)' with 'Georgia'

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Delaware', 'Florida', 'Georgia_(U.S._state)',
       'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas',
       'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts',
       'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana',
       'Nevada', 'New_Hampshire', 'New_Jersey', 'New_Mexico', 'New_York',
       'North_Carolina', 'North_Dakota', 'Ohio', 'Oklahoma', 'Oregon',
       'Pennsylvania', 'Rhode_Island', 'South_Carolina', 'South_Dakota',
       'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West_Virginia', 'Wisconsin', 'Wyoming'], dtype=object)

In [291]:
df_eval['state'] = df_eval['state'].str.replace('Georgia_(U.S._state)','Georgia', regex=False)
df_eval['state'] = df_eval['state'].str.replace('_',' ', regex=False)

### We only need the NAME of the states and the POPESTIMATE2022 for the estimated population in 2022 for each state

In [292]:
pop_est_2022.columns

Index(['SUMLEV', 'REGION', 'DIVISION', 'STATE', 'NAME', 'ESTIMATESBASE2020',
       'POPESTIMATE2020', 'POPESTIMATE2021', 'POPESTIMATE2022', 'NPOPCHG_2020',
       'NPOPCHG_2021', 'NPOPCHG_2022', 'BIRTHS2020', 'BIRTHS2021',
       'BIRTHS2022', 'DEATHS2020', 'DEATHS2021', 'DEATHS2022',
       'NATURALCHG2020', 'NATURALCHG2021', 'NATURALCHG2022',
       'INTERNATIONALMIG2020', 'INTERNATIONALMIG2021', 'INTERNATIONALMIG2022',
       'DOMESTICMIG2020', 'DOMESTICMIG2021', 'DOMESTICMIG2022', 'NETMIG2020',
       'NETMIG2021', 'NETMIG2022', 'RESIDUAL2020', 'RESIDUAL2021',
       'RESIDUAL2022', 'RBIRTH2021', 'RBIRTH2022', 'RDEATH2021', 'RDEATH2022',
       'RNATURALCHG2021', 'RNATURALCHG2022', 'RINTERNATIONALMIG2021',
       'RINTERNATIONALMIG2022', 'RDOMESTICMIG2021', 'RDOMESTICMIG2022',
       'RNETMIG2021', 'RNETMIG2022'],
      dtype='object')

In [293]:
pop_2022 = pop_est_2022[['NAME', 'POPESTIMATE2022']]
pop_2022 = pop_2022.rename(columns={"NAME": "state", "POPESTIMATE2022": "population"})

### Join on the state column to obtain the population field

In [294]:
df_pop = pd.merge(df_eval, pop_2022, on=['state'])
df_pop

,title,lastrevid,score_pred,state,population
0,"Abbeville, Alabama",1171163550,C,Alabama,5074296
1,"Adamsville, Alabama",1177621427,C,Alabama,5074296
2,"Addison, Alabama",1168359898,C,Alabama,5074296
3,"Akron, Alabama",1165909508,GA,Alabama,5074296
4,"Alabaster, Alabama",1179139816,C,Alabama,5074296
...,...,...,...,...,...
21520,"Wamsutter, Wyoming",1169591845,GA,Wyoming,581381
21521,"Wheatland, Wyoming",1176370621,GA,Wyoming,581381
21522,"Worland, Wyoming",1166347917,GA,Wyoming,581381
21523,"Wright, Wyoming",1166334449,GA,Wyoming,581381


### Find out the region each state belongs to and join on state column again

In [297]:
states_by_region = pd.read_csv('US States by Region in Table - US Census Bureau.csv')

In [299]:
states_by_region = states_by_region.rename(columns={"REGION": "regional_division", "STATE": "state"})
states_by_region = states_by_region.drop(columns=['DIVISION'])

In [300]:
df_final = pd.merge(df_pop, states_by_region, on=['state'])
df_final

,title,lastrevid,score_pred,state,population,regional_division
0,"Abbeville, Alabama",1171163550,C,Alabama,5074296,South
1,"Adamsville, Alabama",1177621427,C,Alabama,5074296,South
2,"Addison, Alabama",1168359898,C,Alabama,5074296,South
3,"Akron, Alabama",1165909508,GA,Alabama,5074296,South
4,"Alabaster, Alabama",1179139816,C,Alabama,5074296,South
...,...,...,...,...,...,...
21520,"Wamsutter, Wyoming",1169591845,GA,Wyoming,581381,West
21521,"Wheatland, Wyoming",1176370621,GA,Wyoming,581381,West
21522,"Worland, Wyoming",1166347917,GA,Wyoming,581381,West
21523,"Wright, Wyoming",1166334449,GA,Wyoming,581381,West


In [306]:
df_final = df_final.rename(columns={"title": "article_title", "lastrevid": "revision_id", "score_pred": "article_quality"})

In [315]:
df_final.to_csv('wp_scored_city_articles_by_state.csv', index=False)